### Import libraries

### Connecting to DB


In [12]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from db_connection import start_engine

engine = start_engine()

### Function for pivot table

In [13]:
import pandas as pd
def file_transforming(file, column_name):
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df = df[df["Jahr"] == "Insgesamt"] # Using only 1 column
    df_long = df.melt( # Make a row as a column
        id_vars = ["Jahr"],
        var_name = "year",
        value_name = column_name
    )
    df_long["year"] = pd.to_numeric(df_long["year"], errors="coerce") # Get rid of mistakes with numbers 
    df_long = df_long.dropna(subset=[column_name]) # Drop info with N/A data
    df_final = df_long[df_long["year"] <= 2024] 
    df_final = df_final[["year", column_name]].reset_index(drop=True) # Choosing final data to show 
    return df_final.to_sql(name = column_name, con = engine, # Push to DB
                           if_exists= "replace", index = False)



In [14]:
# Files name
electricity_consumptions = "monatszahlen_energie_und_wasserversorgung_abgabe_an_letztverbraucher_stromverbrauch_25-11-2025.csv"
electricity_generation = "monatszahlen_energie_und_wasserversorgung_elektrizitaetserzeugung_brutto_25-11-2025.csv"
renewable_feed_in = "monatszahlen_energie_und_wasserversorgung_stromeinspeisung_erneuerbare_energien_25-11-2025.csv"
fossil_feed_in = "monatszahlen_energie_und_wasserversorgung_stromeinspeisung_fossile_und_andere_energietraeger_25-11-2025.csv"
total_feed_in = "monatszahlen_energie_und_wasserversorgung_stromeinspeisung_insgesamt_25-11-2025.csv"
# Running functions to push 
file_transforming(electricity_consumptions, "consumed_energie_GWh")
file_transforming(electricity_generation, "generated_energie_GWh")
file_transforming(renewable_feed_in, "renewable_feed_in_GWh")
file_transforming(fossil_feed_in, "fossil_feed_in_GWh")
file_transforming(total_feed_in, "total_feed_in_GWh")

7